In [1]:
import os
os.environ['TFHUB_CACHE_DIR'] = '/home/djjindal/bert/script-learning'
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
import read
import prepare_data
import input_builder
import importlib
import numpy as np
import model
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
from IPython.html import widgets

BATCH_SIZE = 1
LEARNING_RATE = 2e-5
MAX_SEQ_LENGTH = 128

importlib.reload(input_builder)
model_dir = 'output_story'


W0512 15:55:53.080921 139917302007552 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 15:55:56.243963 139917302007552 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/djjindal/venvs/TF-1.3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Model Objects

In [2]:
run_config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_summary_steps=0,
    save_checkpoints_steps=0,
    log_step_count_steps=100)

model_fn = model.model_fn_builder(
  num_labels=2,
  learning_rate=LEARNING_RATE,
  num_train_steps=1,
  num_warmup_steps=1)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'output_story', '_tf_random_seed': None, '_save_summary_steps': 0, '_save_checkpoints_steps': 0, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4083ed9438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0512 15:55:58.502642 139917302007552 tf_logging.py:115] Using config: {'_model_dir': 'output_story', '_tf_random_seed': None, '_save_summary_steps': 0, '_save_checkpoints_steps': 0, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4083ed9438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Some Examples

In [7]:
dataset = list(read.story_read_data_iterator('dataset/story_winter_val.pkl'))
# val_data = dataset[int(0.8 * len(dataset)):]

check_dataset = []
# for i, ec_dict in zip(range(1800), val_data):
#     check_dataset.append(ec_dict)
for ec_dict in dataset:
    check_dataset.append(ec_dict)

predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset,sentences=True,no_context=False, input_size=1800))
predict_input_fn = input_builder.input_fn_builder(
    features=predict_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=2)

predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
print(predictions)
predictions_list = list(predictions)


<generator object Estimator.predict at 0x7f3fecf3b780>
INFO:tensorflow:Calling model_fn.


I0512 15:59:37.825224 139917302007552 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 15:59:41.064236 139917302007552 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0512 15:59:42.616094 139917302007552 tf_logging.py:115] Restoring parameters from output_story/model.ckpt-8300


INFO:tensorflow:Running local_init_op.


I0512 15:59:43.699069 139917302007552 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0512 15:59:43.776803 139917302007552 tf_logging.py:115] Done running local_init_op.


In [8]:
bad = []
for i, c in enumerate(check_dataset):
    r = list(prepare_data.tokenize_if_small_enough([c],sentences=True,no_context=False))
    if not r:
        bad.append(i)
good_gt = [c for i, c in enumerate(check_dataset) if i not in bad]

In [9]:
pred_df = pd.DataFrame.from_dict({'predictions':predictions_list, 'dataset':good_gt})
print(len(good_gt), len(predictions_list))
pred_df['pred_label'] = pred_df.predictions.apply(lambda x: x['labels'])
pred_df['gt_label'] = pred_df.dataset.apply(lambda x: x['correct'])
pred_df['correct_pred'] = pred_df.apply(lambda s: s.pred_label == s.gt_label, axis=1)
print(pred_df.correct_pred.sum()/len(pred_df.index))

1571 1571
0.9548058561425843


In [10]:
pred_df

,predictions,dataset,pred_label,gt_label,correct_pred
0,"{'probabilities': [[-9.894322e-06, -11.5263405...","{'correct': 0, 'sentences': ['Rick grew up in ...",0,0,True
1,"{'probabilities': [[-2.3841855e-07, -15.480089...","{'correct': 0, 'sentences': ['Laverne needs to...",0,0,True
2,"{'probabilities': [[-4.536736, -0.010766041]],...","{'correct': 1, 'sentences': ['Sarah had been d...",1,1,True
3,"{'probabilities': [[-2.8848232e-05, -10.453088...","{'correct': 0, 'sentences': ['Gina was worried...",0,0,True
4,"{'probabilities': [[-1.7881378e-06, -13.211681...","{'correct': 0, 'sentences': ['It was my final...",0,0,True
5,"{'probabilities': [[-1.1324818e-05, -11.383546...","{'correct': 0, 'sentences': ['Jim found an old...",0,0,True
6,"{'probabilities': [[-13.697951, -1.072883e-06]...","{'correct': 1, 'sentences': ['Ron started his ...",1,1,True
7,"{'probabilities': [[-1.1920928e-07, -15.838751...","{'correct': 0, 'sentences': ['John and Billy b...",0,0,True
8,"{'probabilities': [[-14.619933, -4.7683704e-07...","{'correct': 1, 'sentences': ['Caroline was a s...",1,1,True
9,"{'probabilities': [[-4.6491514e-06, -12.282678...","{'correct': 0, 'sentences': ['Trish hated the ...",0,0,True


In [13]:
dataset = list(read.story_read_data_iterator('dataset/story_winter_test.pkl'))
check_dataset = []
for ec_dict in dataset:
    check_dataset.append(ec_dict)

predict_set = list(prepare_data.tokenize_if_small_enough(check_dataset,sentences=True,no_context=False, input_size=1800))
predict_input_fn = input_builder.input_fn_builder(
    features=predict_set,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False,
    candidates=2)

predictions = estimator.predict(input_fn=predict_input_fn,yield_single_examples=False)
print(predictions)
predictions_list2 = list(predictions)


<generator object Estimator.predict at 0x7f3f242ac938>
INFO:tensorflow:Calling model_fn.


I0512 16:08:25.583766 139917302007552 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0512 16:08:29.120226 139917302007552 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0512 16:08:31.085030 139917302007552 tf_logging.py:115] Restoring parameters from output_story/model.ckpt-9000


INFO:tensorflow:Running local_init_op.


I0512 16:08:31.979855 139917302007552 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0512 16:08:32.049348 139917302007552 tf_logging.py:115] Done running local_init_op.


In [39]:
bad = []
for i, c in enumerate(check_dataset):
    r = list(prepare_data.tokenize_if_small_enough([c],sentences=True,no_context=False))
    if not r:
        bad.append(i)
good_gt = [c for i, c in enumerate(check_dataset) if i not in bad]

pred_df = pd.DataFrame.from_dict({'predictions':predictions_list, 'dataset':good_gt})
print(len(good_gt), len(predictions_list))
pred_df['pred_label'] = pred_df.predictions.apply(lambda x: x['labels']+1)
pred_df['gt_label'] = pred_df.dataset.apply(lambda x: x['correct'])

data = pd.read_csv('dataset/cloze_test_test__winter2018-cloze_test_ALL_test - 1.csv', delimiter = ',',names=['InputStoryid','InputSentence1','InputSentence2','InputSentence3','InputSentence4','RandomFifthSentenceQuiz1','RandomFifthSentenceQuiz2','AnswerRightEnding'])
submission = pd.DataFrame.from_dict({'InputStoryid':data['InputStoryid'][1:], 'AnswerRightEnding':pred_df['pred_label'].astype(int)})
    
    

1571 1571


In [57]:
# submission.to_csv("answer.txt", sep=',',index=False)
# data['InputStoryid'] = data['InputStoryid'][1:]
# print(data['InputStoryid'][1:])
# print(pred_df['pred_label'].astype(int))
submission = pd.DataFrame()
data = data.drop(1, axis=0)
submission['AnswerRightEnding'] = pred_df['pred_label'].astype(int)
submission['InputStoryid'] = data['InputStoryid']
print(submission)
# submission = pd.DataFrame.from_dict({'InputStoryid':data['InputStoryid'][1:], 'AnswerRightEnding':pred_df['pred_label'].astype(int)})


KeyError: '[1] not found in axis'